In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import botorch

sns.set(style='whitegrid', font_scale=1.75)

DEVICE = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
import torchsort

import sys
sys.path.append('..')
from experiments.std_bayesopt.helpers import *

In [ ]:
obj_fn = lambda x: np.maximum(-0.125 * x ** 2 + 16 * np.sin(x), 0)

In [ ]:
fig = plt.figure(figsize=(8, 5))

noise_scale = 1.
x_bounds = torch.tensor((-16., 16.)).view(-1, 1)

x = np.linspace(*x_bounds, 64)
f = obj_fn(x)
y = f + noise_scale * np.random.randn(*f.shape)

plt.scatter(x, y, edgecolors='black', facecolors='none', label='observations', s=64, zorder=3)
plt.plot(x, f, color='black', linestyle='--', label='ground truth', linewidth=2, zorder=2)

plt.ylabel('y')
plt.xlabel('x')
plt.ylim((-4, 24))
plt.legend(loc='upper left', ncol=2)

In [ ]:
import botorch
from botorch.fit import fit_gpytorch_model
from botorch.acquisition.analytic import UpperConfidenceBound, ExpectedImprovement
from botorch.optim.optimize import optimize_acqf
from botorch.sampling import IIDNormalSampler

from gpytorch.mlls import ExactMarginalLogLikelihood

In [ ]:
from lambo.utils import DataSplit, update_splits, safe_np_cat
from lambo.optimizers.pymoo import Normalizer

cutoff = np.max(np.where(x < -8))
x_min, y_min = x.min(0), y.min(0)
x_range, y_range = x.max(0) - x_min, y.max(0) - y_min

x_norm = Normalizer(
    loc=x_min + 0.5 * x_range,
    scale=x_range / 2.,
)
y_norm = Normalizer(
    loc=y_min + 0.5 * y_range,
    scale=y_range / 2.,
)

train_x = x[:cutoff]
train_y = y[:cutoff]

all_inputs = torch.tensor(x_norm(x), device=DEVICE).view(-1, 1)
all_targets = torch.tensor(y_norm(y), device=DEVICE).view(-1, 1)
target_dim = all_targets.shape[-1]

new_split = DataSplit(
    all_inputs[:cutoff].cpu().numpy(), all_targets[:cutoff].cpu().numpy()
)
train_split, val_split, test_split = update_splits(
    train_split=DataSplit(),
    val_split=DataSplit(),
    test_split=DataSplit(),
    new_split=new_split,
    holdout_ratio=0.2
)

input_bounds = torch.tensor([-1., 1.], device=DEVICE).view(-1, 1)

In [ ]:
def run_datashift_opt(splits, acqf="ei"):
    def draw_plot(ax):
        # plot p(f | x, D)
        ax.plot(all_inputs.cpu(), f_hat_mean, color='blue', linewidth=2, zorder=4, label='p(f | x, D)')
        ax.fill_between(all_inputs.view(-1).cpu(), f_hat_mean - 1.96 * f_hat_std, f_hat_mean + 1.96 * f_hat_std,
                        color='blue', alpha=0.25)

        # plot a(x)
        ax.plot(all_inputs.cpu(), acq_vals, color='green', zorder=5, linewidth=2, label='a(x)')
        ax.scatter(input_query.cpu(), target_query.cpu(), marker='x', color='red', label='x*', zorder=5,
                   s=32, linewidth=2)

        # plot observed
        ax.scatter(train_inputs.cpu(), train_targets.cpu(), edgecolors='black', facecolors='black',
                   label='D', s=32, zorder=3)

        # plot true function
        ax.plot(all_inputs.cpu(), y_norm(f), color='black', linestyle='--', label='f')

        ax.set_ylim((-2., 2.))

        ax.set_xlabel('x')
        ax.set_ylabel('y')
        return ax

    train_split, val_split, test_split = splits
    
    rx_estimator = None
    optimize_callback = None

    if acqf == 'ei':
        acqf_init = lambda gp, best_f: qExpectedImprovement(
            gp,
            best_f=train_targets.max(0)[0],
            sampler=IIDNormalSampler(64)
        )
    elif acqf == "nei":
        acqf_init = lambda gp, best_f: qNoisyExpectedImprovement(gp, X_baseline=train_inputs, sampler=IIDNormalSampler(64))
    elif acqf == 'conformal_ei':
        def acqf_init(gp, best_f):
            gp.conformal()
            return qConformalExpectedImprovement(
                gp,
                best_f=train_targets.max(0)[0],
                sampler=PassSampler(32),
                cache_root=False
            )

        ######### Ratio Estimator ###########

        class RatioEstimator(nn.Module):
            class _Dataset(torch.utils.data.Dataset):
                def __init__(self):
                    self._prior_ratio = 1
                    self.cls_train_split, self.cls_val_split, self.cls_test_split = update_splits(
                        train_split=DataSplit(),
                        val_split=DataSplit(),
                        test_split=DataSplit(),
                        new_split=DataSplit(),
                        holdout_ratio=0.2
                    )

                def __len__(self):
                    return len(self.cls_train_split.inputs)

                def __getitem__(self, index):
                    return self.cls_train_split.inputs[index], self.cls_train_split.targets[index]

                @property
                def emp_prior(self):
                    return self._prior_ratio

                def _recompute_emp_prior(self):
                    _, train_targets = self.cls_train_split
                    n = len(train_targets)
                    n_p = train_targets.sum().item()
                    if n_p > 0 and n > n_p:
                        self._prior_ratio = n / n_p - 1.
                    return self._prior_ratio

                def _update_splits(self, new_split):
                    self.cls_train_split, self.cls_val_split, self.cls_test_split = update_splits(
                        train_split=self.cls_train_split,
                        val_split=self.cls_val_split,
                        test_split=self.cls_test_split,
                        new_split=new_split,
                        holdout_ratio=0.2)
                    self._recompute_emp_prior()

                def _drop_negative(self):
                    _train_inputs, _train_targets = self.cls_train_split
                    _val_inputs, _val_targets = self.cls_val_split
                    _test_inputs, _test_targets = self.cls_test_split
                    _inputs = safe_np_cat([_train_inputs, _val_inputs, _test_inputs], axis=0)
                    _targets = safe_np_cat([_train_targets, _val_targets, _test_targets], axis=0)
                    mask = np.squeeze(_targets > 0, axis=-1)
                    _inputs, _targets = _inputs[mask, ...], _targets[mask, ...]
                    
                    self.cls_train_split, self.cls_val_split, self.cls_test_split = update_splits(
                        train_split=DataSplit(),
                        val_split=DataSplit(),
                        test_split=DataSplit(),
                        new_split=DataSplit(_inputs, _targets),
                        holdout_ratio=0.2
                    )

                    self._recompute_emp_prior()

            def __init__(self, in_size=1):
                super().__init__()

                self.dataset = RatioEstimator._Dataset()

                ## Remains uniform, when untrained.
                self.classifier = nn.Sequential(
                    nn.Linear(in_size, 1),
                )
                for p in self.classifier.parameters():
                    p.data.fill_(0)
                    p.requires_grad_(True)

                self.criterion = torch.nn.BCEWithLogitsLoss()
                self.optim = torch.optim.Adam(self.classifier.parameters(), lr=1e-2)
            
            @torch.no_grad()
            def forward(self, inputs):
                _p = self.classifier(inputs).squeeze(-1).sigmoid()
                return self.dataset.emp_prior * _p / (1 - _p + 1e-6)

            def optimize_callback(self, xk):
                xk = torch.from_numpy(xk).reshape(-1, 1)
                xk.add_(.1 * torch.randn_like(xk))
                yk = torch.zeros(len(xk), 1)

                self.dataset._update_splits(DataSplit(xk, yk))

                # NOTE: Need to set requires_grad.
                for p in self.classifier.parameters():
                    p.requires_grad_(True)

                if len(self.dataset):
                    loader = torch.utils.data.DataLoader(self.dataset, shuffle=True, batch_size=64)
                    X, y = next(iter(loader))
                    X, y = X.to(DEVICE).float(), y.to(DEVICE)
                    self.optim.zero_grad()
                    loss = self.criterion(self.classifier(X), y)
                    loss.backward()
                    self.optim.step()
        
        rx_estimator = RatioEstimator(in_size=train_split.inputs.shape[-1]).to(DEVICE)
        optimize_callback = rx_estimator.optimize_callback
        
        ######### Ratio Estimator ###########
    elif acqf == "conformal_nei":
        def acqf_init(gp, best_f):
            gp.conformal()
            return qConformalNoisyExpectedImprovement(
                gp,
                X_baseline=train_inputs,
                sampler=PassSampler(32),
                cache_root=False
            )
        
    num_rounds = 32
    plot_interval = 8
    
    queried_targets = []
    for round_idx in range(num_rounds):
        train_inputs = torch.tensor(train_split[0], device=DEVICE)
        train_targets = torch.tensor(train_split[1], device=DEVICE)

        matern_gp = ConformalSingleTaskGP(
            train_X=train_inputs,
            train_Y=train_targets,
            alpha=0.1,
            tgt_grid_res=32,
            conformal_bounds=torch.tensor([[-2., 2.]]).t(),
            ratio_estimator=rx_estimator,
        ).to(DEVICE)
        mll = ExactMarginalLogLikelihood(matern_gp.likelihood, matern_gp)
        fit_gpytorch_model(mll)
        # acq_fn = UpperConfidenceBound(matern_gp, beta=8.)
        # HERE WE USE EI
        # acq_fn = ExpectedImprovement(matern_gp, best_f=train_targets.max())
        acq_fn = acqf_init(matern_gp, train_targets.max())

        matern_gp.requires_grad_(False)
        matern_gp.eval()
        with torch.no_grad():
            f_hat_dist = matern_gp(all_inputs)
            y_hat_dist = matern_gp.likelihood(f_hat_dist)
            f_hat_mean = f_hat_dist.mean.cpu()
            f_hat_std = f_hat_dist.variance.sqrt().cpu()
            y_hat_mean = f_hat_mean.cpu()
            y_hat_std = y_hat_dist.variance.sqrt().cpu()
            acq_vals = acq_fn(all_inputs[:, None]).cpu()

        input_query = optimize_acqf(acq_fn, input_bounds, 1, num_restarts=4, raw_samples=16,
                                    options=dict(callback=optimize_callback))[0]
        x_query = x_norm.inv_transform(input_query.cpu().numpy())
        f_query = obj_fn(x_query)
        y_query = f_query + noise_scale * np.random.randn(*f_query.shape)
        target_query = torch.tensor(y_norm(y_query), device=DEVICE)

        new_split = DataSplit(
            input_query.reshape(-1, 1).cpu(),
            target_query.reshape(-1, 1).cpu(),
        )
        train_split, val_split, test_split = update_splits(
            train_split, val_split, test_split, new_split, holdout_ratio=0.2
        )
        if acqf.startswith('conformal'):
            rx_estimator.dataset._update_splits(
                DataSplit(input_query.reshape(-1, 1).cpu(), torch.ones(1, 1)))
            rx_estimator.dataset._drop_negative()

        queried_targets.append(target_query)

        if round_idx % plot_interval == 0:
            print(f'{train_split[0].shape[0]} train, {val_split[0].shape[0]} val, {test_split[0].shape[0]} test')
            if acqf.startswith('conformal'):
                print(f'[Iterates] {rx_estimator.dataset.cls_train_split[0].shape[0]} train, {rx_estimator.dataset.cls_val_split[0].shape[0]} val, {rx_estimator.dataset.cls_test_split[0].shape[0]} test')
            fig = plt.figure(figsize=(8, 5))
            ax = fig.add_subplot(1, 1, 1)
            draw_plot(ax)

    plt.show()
    return torch.tensor(queried_targets)

In [ ]:
conformal_pts = [run_datashift_opt([train_split, val_split, test_split], acqf="conformal_ei") for _ in range(2)]

In [ ]:
std_pts = [run_datashift_opt([train_split, val_split, test_split], acqf="ei") for _ in range(2)]

In [ ]:
conformal = torch.stack(conformal_pts).cummax(1)[0]
std = torch.stack(std_pts).cummax(1)[0]

In [ ]:
plt.plot(conformal.mean(0), label = "conformal qEI")
plt.fill_between(torch.arange(32), 
                 conformal.mean(0) - 2. / 5**0.5 * conformal.std(0), 
                 conformal.mean(0) + 2. / 5**0.5 * conformal.std(0), 
                 alpha = 0.3)
plt.plot(std.mean(0), label = "qEI")
plt.fill_between(torch.arange(32), 
                 std.mean(0) - 2. / 5**0.5 * std.std(0), 
                 std.mean(0) + 2. / 5**0.5 * std.std(0), 
                 alpha = 0.3)
plt.legend()
plt.xlabel("Function Evaluations")
plt.ylabel("Best Achieved")